Before `ProcessPoolExecutor` let me tell abt `ThreadPoolExecutor`.

Context: Running threads in the background with `loop.run_in_executor`

In [ ]:
import asyncio
import time

def cpu_heavy(n):
    print("Starting heavy work")
    time.sleep(3)
    print("Done heavy work")
    return n * n

async def main():
    loop = asyncio.get_running_loop()
    result = await loop.run_in_executor(None, cpu_heavy, 10)
    print("Result:", result)

asyncio.run(main())

What’s happening:

1.  loop.run_in_executor(None, cpu_heavy, 10)
→ Runs cpu_heavy(10) in a ThreadPoolExecutor (background thread).

2.  The event loop doesn’t block, it can keep serving other coroutines.

3.   await waits for the result, but meanwhile the event loop can switch to other tasks.


By default, `None` means `ThreadPoolExecutor`(best for IO bound tasks). You can also use `ProcessPoolExecutor` for CPU-bound tasks.

`ProcessPoolExecutor` -> Spin up a whole new python process in the background. It’s heavier than threads but bypasses the GIL, so it’s better for CPU-heavy work.

In [ ]:
import asyncio
import time
from concurrent.futures import ProcessPoolExecutor


def cpu_heavy(n):
    print("Starting heavy work")
    time.sleep(3)
    print("Done heavy work")
    return n * n

async def main():
    loop = asyncio.get_running_loop()
    with ProcessPoolExecutor() as pool:
        result = await loop.run_in_executor(pool, cpu_heavy, 10) #Here None is replaced by 'pool'
        print(result)

asyncio.run(main())

Wait!

You might think: 
"Why use ThreadPoolExecutor in the first place since async can handle IO bound tasks gracefully?Also only one thread can execute Python bytecode at a time.
So if you have CPU-bound code (pure Python math loops, JSON parsing, etc.), multiple threads don’t run in parallel, the GIL serializes them??"

Who thinks this long, sry for the very long context😝 Okay, here we go!
There are many libraries and APIs that are blocking and have no async version. For example **requests** (HTTP client), **sqlite3** (database), and standard file operations. These are blocking calls that would block the event loop if used directly in async code. That means even if 100 users hit the server, all are frozen until that blocking call finishes.

> Intuition: Releasing the GIL alone doesn’t create concurrency.

Legacy libraries like requests and sqlite3 are implemented in C and _**release the GIL**_ during blocking operations(If there are no other coroutines to run, releasing the GIL doesn’t help by itself!). This means that while one thread is waiting for a network response or disk I/O, other threads can run Python code.

* Even though the GIL is released during the network wait, in a single-threaded program there’s nothing else to run.
* The program is basically idle while waiting for the response.
* So, releasing the GIL alone doesn’t magically give concurrency, we need other threads or coroutines to make use of that idle time.So using ThreadPoolExecutor with these blocking libraries can improve concurrency and throughput in an async application.

**Qn**:'Why not use ThreadPoolExecutor instead of async then?'

**Ans**: Async is generally more efficient than threads for I/O-bound tasks because it avoids the overhead of thread management and context switching. Async code can handle many concurrent operations with a single thread, while threads require more memory and CPU resources. So use async for I/O-bound tasks when possible, and use ThreadPoolExecutor for blocking libraries that don’t have async versions.


```
Single-thread + blocking I/O
[requests.get()] --> idle (nothing else runs)

ThreadPoolExecutor
[Thread 1: requests.get()] --> GIL released
[Thread 2: another request] --> runs while Thread 1 waits

Async
[Coroutine 1: await aiohttp] --> yields to event loop
[Coroutine 2: await aiohttp] --> runs while Coroutine 1 waits
All in single thread, very lightweight
```

| Work type                                       | Async (`await`) | ThreadPoolExecutor     | ProcessPoolExecutor |
| ----------------------------------------------- | --------------- | ---------------------- | ------------------- |
| Async I/O libs (httpx, aiohttp, asyncpg)        |  Best          |  Not needed           |  Not needed        |
| Blocking I/O libs (requests, sqlite3, file ops) |  Blocks loop   |  Works well           |  Overkill         |
| CPU-heavy (hashing, ML, image resize)           |  Blocks loop   |  GIL prevents speedup |  True parallel     |